In [275]:
import os
import torch
import torchaudio
import torchaudio.sox_effects as sox
import numpy as np
from IPython.display import Audio

In [2]:
sox.init_sox_effects()

/var/tmp/ipykernel_17799/1371593859.py:1: UserWarning: torchaudio.sox_effects.sox_effects.init_sox_effects has been deprecated and will be removed from future release. Please remove the call. This function is called automatically.
  sox.init_sox_effects()


In [3]:
sox.effect_names()

['allpass',
 'band',
 'bandpass',
 'bandreject',
 'bass',
 'bend',
 'biquad',
 'chorus',
 'channels',
 'compand',
 'contrast',
 'dcshift',
 'deemph',
 'delay',
 'dither',
 'divide',
 'downsample',
 'earwax',
 'echo',
 'echos',
 'equalizer',
 'fade',
 'fir',
 'firfit',
 'flanger',
 'gain',
 'highpass',
 'hilbert',
 'loudness',
 'lowpass',
 'mcompand',
 'norm',
 'oops',
 'overdrive',
 'pad',
 'phaser',
 'pitch',
 'rate',
 'remix',
 'repeat',
 'reverb',
 'reverse',
 'riaa',
 'silence',
 'sinc',
 'speed',
 'stat',
 'stats',
 'stretch',
 'swap',
 'synth',
 'tempo',
 'treble',
 'tremolo',
 'trim',
 'upsample',
 'vad',
 'vol']

In [112]:
#Effect params
from torchaudio.utils.sox_utils import list_effects
list_effects()

{'allpass': 'frequency width[h|k|q|o]',
 'band': '[-n] center [width[h|k|q|o]]',
 'bandpass': '[-c] frequency width[h|k|q|o]',
 'bandreject': 'frequency width[h|k|q|o]',
 'bass': 'gain [frequency(100) [width[s|h|k|q|o]](0.5s)]',
 'bend': '[-f frame-rate(25)] [-o over-sample(16)] {start,cents,end}',
 'biquad': 'b0 b1 b2 a0 a1 a2',
 'chorus': 'gain-in gain-out delay decay speed depth [ -s | -t ]',
 'channels': 'number',
 'compand': "attack1,decay1{,attack2,decay2} [soft-knee-dB:]in-dB1[,out-dB1]{,in-dB2,out-dB2} [gain [initial-volume-dB [delay]]]\n\twhere {} means optional and repeatable and [] means optional.\n\tdB values are floating point or -inf'; times are in seconds.",
 'contrast': '[enhancement (75)]',
 'dcshift': 'shift [ limitergain ]\n\tThe peak limiter has a gain much less than 1.0 (ie 0.05 or 0.02) which\n\tis only used on peaks to prevent clipping. (default is no limiter)',
 'deemph': '',
 'delay': '{position}',
 'dither': '[-S|-s|-f filter] [-a] [-p precision]\n  (none)   U

In [374]:
waveform, sample_rate = torchaudio.load('../datasets/GTZAN/gtzan_genre/genres/disco/disco.00013.wav')
waveform = waveform.mean(dim=0, keepdim=True)  # convert stereo to mono
Audio(waveform, rate=sample_rate)

--2023-03-22 11:18:47--  https://www.dropbox.com/s/2x1q7q7q7q7q7q7/1.wav?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘1.wav’

1.wav                   [ <=>                ]  48.78K  --.-KB/s    in 0.001s  

2023-03-22 11:18:47 (53.9 MB/s) - ‘1.wav’ saved [49947]



In [372]:
gain = np.random.randint(-12, 0)
pitch = np.random.randint(-1200, +1200)
reverberance, damping_factor, room_size = (np.random.randint(0, 100),) * 3

gain_in = round(np.random.uniform(0.1, 1.0), 1)
gain_out = round(np.random.uniform(0.1, 1.0), 1)
decay = round(np.random.uniform(0.1, 0.9),1)
ch_speed = round(np.random.uniform(0.1, 2.0),2)
depth = np.random.randint(2, 5)
mode = np.random.choice(["-s", "-t"])

drive = np.random.randint(0, 30)

stretch = round(np.random.uniform(0.8, 1.2), 1)
speed = np.random.uniform(0.5, 1.5)

tremolo_speed = np.random.uniform(0.1, 100)  # Choose a random value between 0.1 and 100 Hz
tremolo_depth = np.random.randint(1, 101)  # Choose a random value between 0% and 100%

effects = [
    ["gain","-n",str(gain)],  # apply 10 db attenuation
    ["chorus",str(gain_in),str(gain_out),"55",str(decay),str(ch_speed),str(depth),mode],  # chorus effect
    ["overdrive", str(drive)],  #'overdrive': '[gain [colour]]',
    ["pitch",str(pitch)],  #'pitch': 'semitones [octaves [cents]]',
    [
        "reverb",
        str(reverberance),
        str(damping_factor),
        str(room_size),
    ],  #'reverb': '[-w|--wet-only] [reverberance (50%) [HF-damping (50%) [room-scale (100%) [stereo-depth (100%) [pre-delay (0ms) [wet-gain (0dB)]]]]]]',
    ["speed", str(speed)],
    ["stretch", str(stretch)],  #'stretch': 'factor [window fade shift fading]\n       (expansion, frame in ms, lin/..., unit<1.0, unit<0.5)\n       (defaults: 1.0 20 lin ...)',
    ["tremolo", str(tremolo_speed), str(tremolo_depth)],  #'tremolo': 'speed_Hz [depth_percent]',
]
positive, _ = sox.apply_effects_tensor(waveform, sample_rate, effects)
positive = positive.mean(dim=0, keepdim=True)  # convert stereo to mono

In [373]:
positive, sample_rate = sox.apply_effects_tensor(waveform, sample_rate, effects)
Audio(waveform, rate=sample_rate)